# Introduction
Companies that deal with a large volume of products - such as distributors, industries and retail chains - often face the challenge of identifying products based on inaccurate, incomplete or varied textual descriptions. In environments where data is entered manually, typos, abbreviations and different trade names can make it difficult to correctly identify items in systems such as Enterprise Resource Planning (ERP), Customer Relationship Management (CRM) and e-commerce platforms.

In this scenario, there is a common need for tools that can:

Interpret informal or incorrect descriptions provided by users.

Suggest the most similar products based on semantic similarity.

Guarantee a fallback with traditional algorithms (such as fuzzy matching) if the semantic search does not find relevant results.

Integrate with APIs and automated flows of intelligent agents.

In this tutorial, we will learn how to create an AI agent specialized in resolving inconsistencies in customer return invoices. The agent is able to interact with an MCP server that provides vector search and invoice retrieval tools, allowing the agent to automatically find the company's original A/R invoice based on information provided by the customer.

A MCP Server (Model Context Protocol) is a software component responsible for mediating communication between intelligent agents and external services through a standardized protocol of messages and tools. It acts as a bridge between language models (LLMs) and existing APIs, allowing AI agents to access specific functionalities of legacy systems without the need to rewrite or directly adapt them.

### How does it work?

The MCP server:

Exposes a set of named tools that represent actions or services agents can call.
Each tool can be connected to a database function, REST API, legacy system, or any programmable operation.
Receives agent commands (in JSON format), executes the corresponding service, and returns a standardized response.

### Usage with Legacy Applications

Legacy applications often already provide functionality through databases, files, or internal APIs. With a MCP Server, you can:

Encapsulate calls to REST services, stored procedures, or SQL queries.

Make these services accessible to an agent through a unified communication interface.

Integrate older systems with modern architectures based on conversational AI and RAG.

### Advantages

Reuse of existing business logic.

No need to refactor legacy systems.

Enables LLM agents to interact with external systems in a secure and controlled manner.

Facilitates testing, versioning, and modularity in the integration between AI and backend services.

## Architecture

This implementation uses:

- **Google Generative AI** (Gemini 2.0 Flash) - Language model and embeddings for semantic search
- **Oracle Database 23ai (Free)** - Local or Oracle Free Cloud database for data storage and vector indexing
- **MCP Server** (FastMCP) - Single stdio-based server providing three tools for invoice resolution
- **Custom Agent Loop** - Pure Python ReAct agent implementation with Gemini API

This modular design eliminates unnecessary framework dependencies (no LangChain/LangGraph required) and leverages Gemini's native tool-calling capabilities for efficient agent orchestration.

# Objectives
Build a completely independent MCP-based agent without LangChain or LangGraph dependencies.

Integrate the agent with a local MCP server using the stdio protocol.

Use an MCP server with three tools:
- Perform vector searches from product descriptions using Gemini embeddings.
- Identify the most likely EAN code for an item using advanced SQL search.
- Search for original invoices based on customer, state, EAN, and price criteria.

Simulate a real problem resolution based on a JSON input such as:
```json
{
  "customer": "Customer 43",
  "description": "Harry Potter",
  "price": 139.55,
  "location": "RJ"
}
```

### Prerequisites

* **Python 3.12 or higher** - Required for async support and latest features.

* **Google Cloud Account** - For access to Generative AI (free tier available).
  - Create a project at console.cloud.google.com
  - Enable the Generative AI API
  - Generate an API key at https://aistudio.google.com/app/apikey

* **Oracle Database (Free)** - One of:
  - Oracle Database 23ai Developer Release (local installation)
  - Oracle Free SQL Cloud (cloud-based, no credit card required)
  - Connection string format: `host:port/service_name` (e.g., `db.freesql.com:1521/23ai`)

* **Install auxiliary libraries via pip install -r requirements.txt:**
  - `google-generativeai` - Google's LLM and embedding API
  - `mcp` - Model Context Protocol
  - `mcp-server-fastmcp` - FastMCP for MCP server
  - `oracledb` - Oracle database driver (thin mode, no Oracle client required)
  - `numpy` - Numerical operations
  - `rapidfuzz` - Fuzzy string matching fallback
  - `python-dotenv` - Environment variable management

* **A functional MCP server** with three tools:
  - `search_vectorized_product` - Semantic product search using embeddings
  - `resolve_ean` - EAN code resolution via advanced search
  - `search_invoices_by_criteria` - Invoice lookup by customer, state, EAN, and price

* **Configuration Files:**
  - Create a `.env` file in the project root with:
    ```
    GOOGLE_API_KEY=your_google_api_key_here
    ORACLE_DSN=host:port/service_name
    ORACLE_USER=your_username
    ORACLE_PASSWORD=your_password
    ```
  - Run `pip install -r requirements.txt` to install all dependencies

# Task 1: Set Up Local Oracle Database 23ai (Free)

This project supports multiple Oracle database options:

## Option A: Local Installation
Install Oracle Database 23ai Developer Release locally on your machine (suitable for development/testing).

## Option B: Oracle Free SQL Cloud (Recommended)
Use Oracle's free cloud database offering at https://www.oracle.com/cloud/free/

### Steps for Oracle Free SQL Cloud:

1. Sign in or create a free Oracle Cloud account at oracle.com/cloud/free

2. Navigate to Databases → Autonomous Database (Free Tier)

3. Click "Create Autonomous Database Instance"

4. Configure:
   - **Database Name**: Name your instance (e.g., `mcp_oracle`)
   - **Workload Type**: Select "Transaction Processing"
   - **Deployment**: Transaction Processing (Serverless)
   - **Always Free**: Ensure "Always Free" tier is selected

5. Create a strong ADMIN password

6. Click "Create Autonomous Database" (wait a few minutes for provisioning)

7. Once ready, click the database name to open its details

8. Click "Database Connection" to get the connection details

9. For simple connections, use the Easy Connect format:
   ```
   hostname:port/service_name
   ```

10. Download the database wallet if you prefer wallet-based authentication (optional)

### Connection Testing

Create a `.env` file in your project root:
```
GOOGLE_API_KEY=your_key_from_google_cloud
ORACLE_DSN=your_hostname:1521/your_service_name
ORACLE_USER=admin
ORACLE_PASSWORD=your_password_here
```

Test the connection by running:
```bash
python -c "
import oracledb
import os
from dotenv import load_dotenv

load_dotenv()
conn = oracledb.connect(
    user=os.getenv('ORACLE_USER'),
    password=os.getenv('ORACLE_PASSWORD'),
    dsn=os.getenv('ORACLE_DSN')
)
print('✅ Successfully connected to Oracle Database')
conn.close()
"
```

# Task 2: Run the Autonomous Database Table Creation Script
Prepare the database for our use case. Download and run the SQL script from here: script.sql that creates three essential tables (PRODUCTS, INVOICE and ITEM_INVOICE) for the scenario of reconciling invoices with AI agents.

1. Go to the OCI Console, navigate to Oracle Database, Autonomous Database and click the name of the newly created instance.

2. Click Database Actions and SQL to open the SQL console in the browser.

3. Download and open the script.sql file locally and paste all the contents into the SQL console editor.

4. Click Run or press Ctrl + Enter. Wait for confirmation that the commands have been executed successfully.

5. Run the following commands to verify that the tables have been created.

    ```bash
    SELECT table_name FROM user_tables;
    ```  

# Task 3: Insert Example Data into Tables
Insert dummy data that will simulate a real scenario for the application of AI agents. We will use two SQL scripts:

* insert_products_books.sql: Inserts a list of books as products, with their respective EANs and descriptions.
* invoice_data_insert.sql: Inserts mock A/R invoice records, associated with customers, products and prices.

This data will be used by AI agents to resolve inconsistencies in returns invoices.

1. Go to the OCI Console, navigate to Oracle Database, Autonomous Database and click the name of the newly created instance.

2. Click Database Actions and SQL to open the SQL console in the browser.

3. Download the insert_products_books.sql file content from here: insert_products_books.sql and paste it into the SQL editor.

4. Click Run or press Ctrl + Enter.

5. Now, download the invoice_data_insert.sql file content from here: invoice_data_insert.sql file and paste it into the editor.

6. Click Run or press Ctrl + Enter.

7. Run the following command to check the data.
    ```bash
    SELECT * FROM PRODUCTS;
    SELECT * FROM INVOICE;
    SELECT * FROM ITEM_INVOICE;
    ```

# Task 4: Create and Compile the Advanced Search Function in the Database
Create a PL/SQL function called fn_advanced_search, which performs intelligent searches for keywords in product descriptions. This function will be used by AI agents as part of the resolve_ean tool, allowing them to find the nearest EAN code based on the description provided by a customer on the returns note.

What does the function do?

* Tokenization of input terms (Example: harry poter stone becomes [harry, poter, stone])

* Search techniques with scoring:

    * Direct search in descriptions (LIKE '%term%') → +3 points.

    * Phonetic search using SOUNDEX → +2 points.

    * Similar spelling search using UTL_MATCH.EDIT_DISTANCE <= 2 → +1 point.

* Scoring logic:

    * Sums points for each product.
    * Returns only products with total score > 0.
* Return format:

    Products are returned as result_product type objects containing:

    * code (EAN).
    * description (product description).
    * similarity (calculated search score).

Follow the steps:

1. Copy and paste the complete similarity_search.sql script into the autonomous database SQL console.

    This includes:

    * Creating the products table (if it has not already been done).
    * Creating a text index.
    * Types product_result and product_result_tab.
    * The fn_advanced_search function.
    * Optional tests.
2. Run the complete script. The result should be Function created and Type created.

3. Run the following query to test the function with simulated descriptions.
    ```bash
    SELECT *
    FROM TABLE(fn_advanced_search('harry poter askaban'))
    ORDER BY similarity DESC;
    ```

# Task 5: Vectorize Products for Semantic Search with AI
In this task, we will complement advanced SQL-based search with a new approach based on semantic vectors. This will be especially useful for AI agents that use embeddings (numerical representations of phrases) to compare similarity between product descriptions - more flexibly and intelligently than word or phonetic searches.

To do this, we will use the Python script (process_vector_products.py), which connects to the Oracle database, extracts the products from the PRODUCTS table, transforms their descriptions into vectors (embeddings), and builds a vector index using the Oracle database itself.

### What does the script do?

* Reads the products from the products table using oracledb.
* Generates the embeddings using the all-MiniLM-L6-v2 model from the sentence-transformers package.
* Create the embeddings_products table to store the vectors directly in Oracle.
* Insert or update the records and save the vector as a binary BLOB (in serialized float32 format).

**Note**: The embeddings are converted into bytes with np.float32.tobytes() to be stored as a BLOB. To retrieve the vectors, use np.frombuffer(blob, dtype=np.float32).

This format allows future similarity searches to be done directly using SQL or by loading the vectors from the database for operations with np.dot, cosine_similarity or integration with LLMs.

This script generates semantic embeddings for products and writes these vectors to the Oracle Database 23ai. The main points are highlighted below:

* Configure the Connection to Oracle using Wallet.

The code uses the oracledb library in thin mode and configures secure access using an Oracle Wallet.
```bash
os.environ["TNS_ADMIN"] = WALLET_PATH
connection = oracledb.connect(
    user=USERNAME,
    password=PASSWORD,
    dsn=DB_ALIAS,
    ...
)
```
* Consultation of the Product Table.

The products table contains the original data (ID, code and description). These descriptions are used as the basis for generating the semantic vectors.
```bash
cursor.execute("SELECT id, code, description FROM products")
```
* Generate Embeddings with sentence-transformers

The all-MiniLM-L6-v2 model is used to transform product descriptions into high-dimensional numerical vectors.
```bash
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(descriptions, convert_to_numpy=True)
```

* Create the Embeddings Table (if it does not exist).

    The embeddings_products table is created dynamically with the following fields:

    * id: Product identifier (primary key).
    * code: Product code.
    * description: Original description.
    * vector: BLOB containing the vector serialized in float32.
    ```bash
    CREATE TABLE embeddings_products (
    id NUMBER PRIMARY KEY,
    code VARCHAR2(100),
    description VARCHAR2(4000),
    BLOB vector
    )
    ```

Note: The creation uses EXECUTE IMMEDIATE inside a BEGIN...EXCEPTION to avoid an error if the table already exists.

* Insert or Update using MERGE.

    For each product, the vector is converted into bytes (float32) and inserted or updated in the embeddings_products table using a MERGE INTO.
    ```bash
    vector_bytes = vector.astype(np.float32).tobytes()
    MERGE INTO embeddings_products ...
    ```

Run the Script:

**Note**: You need Oracle Wallet downloaded and configured.

Run it in the terminal.
```bash
python process_vector_products.py
```
Done! The products in the database have been vectorized.

Why is this important?

Vector search is highly effective for finding products even when the description is subjective, imprecise or in natural language.

# Understand the Code: LLM Agent with MCP Server
This project is made up of 3 main components:

* **ReAct Agent with Custom Loop and Gemini** (main.py) - Pure Python agent without LangChain/LangGraph
* **MCP Server with Invoice Resolution Tools** (server_invoice_items.py) - FastMCP-based server
* **Product Search with Gemini Embeddings** (product_search.py) - Semantic search with vector database

## Component Details:

### 1. ReAct Agent with Gemini (main.py)

This component runs the main application, where the user interacts with an agent powered by Google's Gemini model. It communicates with the MCP server using the stdio protocol.

Main features:

* **Native Gemini Integration** - No LangChain wrapper, direct API calls
  ```python
  model = genai.GenerativeModel(
      model_name="gemini-2.0-flash",
      system_instruction=SYSTEM_PROMPT,
      tools=TOOLS_SCHEMA
  )
  ```

* **Custom ReAct Agent Loop** - Pure Python implementation with no framework overhead
  - Gets response from Gemini with tool calls
  - Parses function calls from response
  - Executes MCP tools
  - Sends results back to Gemini
  - Repeats until final response

* **Direct MCP Server Connection** - Simple stdio client
  ```python
  async with stdio_client(server_params) as client:
      await client.initialize()
  ```

* **Prompt Caching** - Reduces token usage for repeated system prompts (Gemini feature)

* **Main User Interaction Loop**
  ```python
  while True:
      query = input("You: ")
      await run_agent_loop(client, query, memory_state)
  ```

### 2. MCP Server with Resolution Tools (server_invoice_items.py)

This server responds to agent calls by providing tools that access an Oracle Database with product and invoice information.

Main features:

* **FastMCP Server** - Lightweight MCP implementation
  ```python
  mcp = FastMCP("InvoiceItemResolver")
  ```

* **Connection to Oracle Database** - Using oracledb (thin mode, no wallet required for local setup)
  ```python
  connection = oracledb.connect(
      user=USERNAME,
      password=PASSWORD,
      dsn=DB_DSN
  )
  ```

* **Three MCP Tools**:

  * **search_vectorized_product**: Searches for similar products using semantic embeddings
    ```python
    @mcp.tool()
    def search_vectorized_product(description: str) -> dict:
        """Searches for a product by description using embeddings."""
        return searcher.search_similar_products(description)
    ```

  * **resolve_ean**: Resolves an EAN based on description similarity using advanced SQL
    ```python
    @mcp.tool()
    def resolve_ean(description: str) -> dict:
        result = execute_ean_search(description)
        return {"ean": result[0]["code"], ...}
    ```

  * **search_invoices_by_criteria**: Searches for A/R invoices based on filters
    ```python
    @mcp.tool()
    def search_invoices_by_criteria(customer=None, state=None, price=None, ean=None):
        """Searches for outbound invoices by customer, state, EAN, and price"""
        # Constructs dynamic query based on provided fields
    ```

* **Runs in stdio Mode**
  ```python
  if __name__ == "__main__":
      mcp.run(transport="stdio")
  ```

### 3. Product Search with Gemini Embeddings (product_search.py)

This module implements semantic product search using Google's embedding model and Oracle vector storage.

Key methods:

* **Initialize with Gemini embeddings** - Uses `models/embedding-001`
* **Load vectors from database** - Retrieves pre-generated embeddings from Oracle
* **Semantic search** - Computes cosine/Euclidean distance to find similar products
* **Fuzzy matching fallback** - Uses RapidFuzz if semantic search fails

## Key Differences from Original Setup

| Aspect | Original (OCI) | New (Gemini + Local) |
|--------|---|---|
| **LLM** | Cohere (OCI Generative AI) | Gemini 2.0 Flash |
| **Agent Framework** | LangGraph + LangChain | Custom ReAct loop |
| **Database** | Autonomous DB (Cloud) | Oracle Free (Local or Cloud) |
| **Embeddings** | OCI Cohere Embeddings | Google Embeddings API |
| **Server Framework** | FastMCP with LangChain adapters | Pure FastMCP |
| **Observability** | Phoenix + OpenTelemetry | (Optional, removed by default) |
| **Dependencies** | 12+ packages | 6 packages (minimal) |
| **Cost** | Pay for OCI services | Free tier for Gemini + Oracle |

# Task 6: Configure Google Generative AI

Configure the language model and embeddings used by the conversational agent based on the MCP protocol, using Google's Generative AI services.

## 1. Set Up Google Cloud Project

1. Go to https://console.cloud.google.com
2. Create a new project or use existing one
3. Enable the "Generative AI API"
4. Go to https://aistudio.google.com/app/apikey
5. Click "Create API Key"
6. Copy the API key

## 2. Configure Environment Variables

Create a `.env` file in your project root:

```bash
GOOGLE_API_KEY=your_api_key_here
ORACLE_DSN=localhost:1521/xe
ORACLE_USER=admin
ORACLE_PASSWORD=your_password
```

The configurations are already in the code:

### Language Model Configuration (main.py)

```python
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    system_instruction=SYSTEM_PROMPT,
    tools=TOOLS_SCHEMA
)

response = model.generate_content(
    messages,
    generation_config=genai.types.GenerationConfig(
        temperature=0.1,
        max_output_tokens=2000
    )
)
```

### Embeddings Configuration (product_search.py)

The embeddings are automatically generated using Google's embedding model:

```python
response = genai.embed_content(
    model="models/embedding-001",
    content=text,
    task_type="SEMANTIC_SIMILARITY"
)
embedding = np.array(response['embedding'])
```

### Running the Embedding Generation Script

Run the process_vector_products.py script to generate and store embeddings:

```bash
python process_vector_products.py
```

This script will:
- Connect to your Oracle database
- Read all products from the PRODUCTS table
- Generate embeddings using Google's API
- Store vectors in the embeddings_products table

**Output:**
```
============================================================
📊 Product Vector Embedding Generator
============================================================
🔌 Database: localhost:1521/xe
👤 User: admin
🤖 Embedding Model: models/embedding-001
============================================================
✅ Connected to Oracle Database
✅ Retrieved 50 products from database

📈 Generating embeddings...
   ✓ Embedded 10/50 products
   ✓ Embedded 20/50 products
   ✓ Embedded 30/50 products
   ✓ Embedded 40/50 products
   ✓ Embedded 50/50 products
✅ Generated embeddings for 50 products

📋 Creating embeddings table...
✅ Embeddings table ready (created or already exists)

💾 Storing embeddings in database...
✅ Stored 50 embeddings successfully

============================================================
✅ Vector embedding process completed successfully!
============================================================
```

## 3. Verify Configuration

Test that everything is working:

```bash
# Test Google API access
python -c "
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
if api_key:
    genai.configure(api_key=api_key)
    print('✅ Google API configured')
else:
    print('❌ GOOGLE_API_KEY not found')
"

# Test Oracle connection
python -c "
import oracledb
import os
from dotenv import load_dotenv

load_dotenv()
conn = oracledb.connect(
    user=os.getenv('ORACLE_USER'),
    password=os.getenv('ORACLE_PASSWORD'),
    dsn=os.getenv('ORACLE_DSN')
)
print('✅ Oracle connection successful')
conn.close()
"
```

With this, the LLM model and embeddings are ready to be used by the MCP agent.

# Task 7: Test the Product and Invoice Description Search
1. Run the main.py file using the following command.
```bash
    python main.py
```
2. When the You: prompt appears, enter the following statement.
    ```json
    { "customer": "Customer 108",  "description": "Harry Poter askaban", "price": 82.26, "location": "SP"}
    ```
    Note: The name of the book Harry Potter and Azkaban are spelled wrong, but the engine can find it without any problems.

    Note that the services have been executed:
    ```bash
    fetch_vectorized_product
    resolve_ean
    search_invoices_by_criteria
    ```
3. Now, enter the following statement.
    ```json
    { "customer": "Customer 108",  "description": "Harry Poter askaban", "price": 82.26}
    ```
    You will see that there was no invoice record found. This is because location is key to finding an invoice.

4. Now, enter the following statement.
    ```json
    { "customer": "Customer 108",  "description": "Harry Poter", "location": "SP"}
    ```
    This time, we enter the location but omit the unit price.
    And yet the invoice was found. This is because the price is not fundamental, but it does help to close the gap to be more assertive.

    Some examples for testing are:
    ```json
        { "customer": "Customer 108",  "description": "Harry Poter askaban", "price": 82.26, "location": "SP"}

        { "customer": "Customer 108",  "description": "Harry Poter askaban", "price": 82.26}

        { "customer": "Customer 108",  "description": "Harry Poter askaban", "location": "SP"}

        { "customer": "Customer 108",  "description": "Harry Poter askaban", "price": 82.26, "location": "RJ"}

        { "customer": "Customer 125",  "description": "Harry Potter Chamber", "price": 79.16, "location": "SP"}

        { "customer": "Customer 125",  "description": "Harry Potter Chamber", "price": 79.15, "location": "SP"}
    ```


# Task 8: Optional - View Observability with Phoenix

**Note**: Phoenix observability is OPTIONAL and not required for the agent to work. This setup is stripped down for simplicity.

If you want to enable observability:

1. Install Phoenix: `pip install arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp`

2. Update `main.py` to add the Phoenix initialization code (see comments in original file)

3. Start Phoenix: `phoenix.launch_app()`

4. View traces at `http://localhost:6006/`

For now, the agent logs all tool calls and responses to the console, which is sufficient for debugging and monitoring.

In [ ]:
import oracledb
import os
from dotenv import load_dotenv

load_dotenv()

# Test connection to Oracle Database
try:
    dsn = os.getenv("ORACLE_DSN", "localhost:1521/xe")
    user = os.getenv("ORACLE_USER", "admin")
    password = os.getenv("ORACLE_PASSWORD", "")
    
    connection = oracledb.connect(
        user=user,
        password=password,
        dsn=dsn
    )
    
    print(f"✅ Successfully connected to Oracle Database at {dsn}")
    
    cursor = connection.cursor()
    for result in cursor.execute("SELECT * FROM DUAL"):
        print(f"   Result: {result}")
    
    connection.close()
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print(f"   Check your .env file with ORACLE_DSN, ORACLE_USER, and ORACLE_PASSWORD")

In [ ]:
import oracledb
# import os
# from dotenv import load_dotenv

# load_dotenv()

print("Before init...")

oracledb.init_oracle_client(
    lib_dir=r"C:\oracle\instantclient_23_0"
)

print("OCI loaded successfully")

local_dsn = "db.freesql.com:1521/23ai_34ui2"
connection = oracledb.connect(
    user="LOKHANDEJAY90_SCHEMA_QYEJV",
    password="FRNN2376P5!UUIVkHYX701M9R0C5VW",
    dsn=local_dsn,
    disable_oob=True)

print("Successfully connected to Oracle Database")

cursor = connection.cursor()
for result in cursor.execute("SELECT * FROM HR.COUNTRIES"):
    print(result)
